In [1]:
import numpy as np
import pandas as pd
import glob
import pygrib
#%matplotlib inline
import matplotlib.pyplot as plt
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import cartopy.crs as ccrs
import datetime as dt
from IPython.display import clear_output

In [2]:
def MonthNumberToMonthName(month):
    date = dt.date(2000, month, 1)  #年と日は自由に設定する
    return date.strftime("%B")

dtg, from lon lat, eeid無し

時間はdt_pos_utc使う

In [3]:
path = "D:/AIS/one-year-data/pctc-data"
allfiles = glob.glob(path + "/*.csv") 
frame = pd.DataFrame()
list_ = []
for file_ in allfiles:
    DF = pd.read_csv(file_,encoding='UTF-8',usecols=["mmsi","imo","vessel_name","length","width",
                             "flag_country","destination","eta",
                             "draught", "longitude","latitude","draught",
                             "sog","cog","rot","heading",
                             "ts_pos_utc","ts_static_utc","dt_pos_utc","dt_static_utc"]) 
    DF['longitude'] = pd.to_numeric(DF['longitude'],errors='coerce')
    DF['latitude'] = pd.to_numeric(DF['latitude'],errors='coerce')
    list_.append(DF)
frame = pd.concat(list_, join='inner')

frame["dt_pos_utc"] = pd.to_datetime(frame["dt_pos_utc"],format="%Y-%m-%d %H:%M:%S")
print(frame)
print(frame.info())

          mmsi        imo  vessel_name  length  width flag_country  \
0    205451000  9125384.0   CLEMENTINE   162.0   25.0      Belgium   
1    205451000  9125384.0   CLEMENTINE   162.0   25.0      Belgium   
2    205451000  9125384.0   CLEMENTINE   162.0   25.0      Belgium   
3    205451000  9125384.0   CLEMENTINE   162.0   25.0      Belgium   
4    205451000  9125384.0   CLEMENTINE   162.0   25.0      Belgium   
..         ...        ...          ...     ...    ...          ...   
746  775353000  8009088.0  LA CARACOLA   121.0   22.0    Venezuela   
747  775353000  8009088.0  LA CARACOLA   121.0   22.0    Venezuela   
748  775353000  8009088.0  LA CARACOLA   121.0   22.0    Venezuela   
749  775353000  8009088.0  LA CARACOLA   121.0   22.0    Venezuela   
750  775353000  8009088.0  LA CARACOLA   121.0   22.0    Venezuela   

         destination      eta  draught  longitude   latitude   sog    cog  \
0            ESBJERG  8020800      6.1   3.211667  51.341667   0.0  185.0   
1    

In [4]:
def plot_func(year, month, day):   
#3時間に含まれるAISデータをframe2に格納
    frame2 = frame[(frame["dt_pos_utc"]>=dt.datetime(year, month, day)) & (frame["dt_pos_utc"]<dt.datetime(year,month,day)+dt.timedelta(days=1))]
#AISをプロット
    fig = plt.figure(figsize=(160,90), dpi=12)

    ax = fig.add_subplot(  1, 1, 1, projection=ccrs.PlateCarree(central_longitude=0))
    ax.set_global()
    ax.stock_img()
    ax.coastlines()

    title = "PCTC    " + "{} {}, {}".format(MonthNumberToMonthName(month),day,year)
    ax.set_title(title, size=250)

    X1 = frame2[frame2["length"]<100].longitude
    Y1 = frame2[frame2["length"]<100].latitude
    plt.scatter(X1,Y1, c='red', label="below 100m" ,s=100)

    X2 = frame2[(frame2["length"]>=100) & (frame2["length"]<200)].longitude
    Y2 = frame2[(frame2["length"]>=100) & (frame2["length"]<200)].latitude
    plt.scatter(X2,Y2, c='orange', label="100m ~ 200m" ,s=100)

    X3 = frame2[(frame2["length"]>=200) & (frame2["length"]<300)].longitude
    Y3 = frame2[(frame2["length"]>=200) & (frame2["length"]<300)].latitude
    plt.scatter(X3,Y3, c='yellow', label="200m ~ 300m" ,s=100)

    X4 = frame2[frame2["length"]>=300].longitude
    Y4 = frame2[frame2["length"]>=300].latitude
    plt.scatter(X4,Y4, c='olive', label="over 300m" ,s=100)

    plt.legend(frameon=False, markerscale = 10, bbox_to_anchor=(1.01,0.14), loc="upper left", borderaxespad=0, fontsize=100)
    
    fig.savefig("D:/AIS/one-year-data/plot/pctc/{}-{}/PCTC_{}-{}-{}.jpg".format(str(year).zfill(4), str(month).zfill(2),  str(year).zfill(4), str(month).zfill(2), str(day).zfill(2)))
    fig.savefig("D:/AIS/one-year-data/plot/pctc/all/PCTC_{}-{}-{}.jpg".format(str(year).zfill(4), str(month).zfill(2), str(day).zfill(2)))
        
    plt.close("all")
    clear_output(True)
        
    print("{}-{}-{} saved!".format(str(year).zfill(4), str(month).zfill(2), str(day).zfill(2)))        

In [5]:
date = dt.datetime(2018,9,1)
for i in range(0,365):
    plot_func(date.year,date.month,date.day)
    clear_output(True)
    date += dt.timedelta(days=1)

2019-08-31 saved!


In [6]:
import glob
from PIL import Image

path = "D:/AIS/one-year-data/plot/pctc/all"
images = sorted(glob.glob(path + "/*.jpg"))
frames = []
print(len(images))

for image in images:
    new_frame = Image.open(image)
    frames.append(new_frame)
    
frames[0].save(path+"/pctc_one-year.gif", append_images=frames[1:], save_all=True, duration=100, loop=0)

365
